# Haiku


In [131]:
import sqlite3
import requests  # To get the pages
import hashlib
import re

import pyphen
import syllapy
from collections import defaultdict, Counter

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

from statistics import mean
# from bs4 import BeautifulSoup # and to process them

con = sqlite3.connect("spam.db")
cur = con.cursor()


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/parallels/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Globals

In [4]:
base_url = "http://web.mit.edu/jync/www/spam/"
extension = ".html"

## Scraper Table

In [5]:
st = cur.execute("CREATE TABLE IF NOT EXISTS \"url_cache\" (\
	\"hash\"	TEXT,\
	\"text\"	TEXT\
);")


## Poem Table

In [6]:
# pt = cur.execute("DROP TABLE poems;")

pt = cur.execute("CREATE TABLE IF NOT EXISTS \"poems\" (\
	\"id\"	INTEGER UNIQUE,\
	\"line_1\"	TEXT,\
	\"line_2\"	TEXT,\
	\"line_3\"	TEXT,\
	\"author\"	TEXT\
);")
print(pt)

# Scraper

In [7]:
for i in range ( 1001,19601,100 ) :
    url = base_url + str(i) + "-" + str(i + 99) + extension
    # print(url)
    
    ID = i
    hash = hashlib.md5(url.encode()).hexdigest()
    
    # https://stackoverflow.com/a/9756276
    res = cur.execute("SELECT EXISTS(SELECT 1 FROM url_cache WHERE hash='"+hash+"');").fetchone()[0]
    
    if res is 0 :
        r = requests.get(url)
        text = r.text
        cur.execute("INSERT INTO url_cache VALUES ( ?, ?)", ( hash, text ) )
        # print(hash)

    else :
        # print(hash)
        continue 
        
con.commit()

## Poem Extractor

In [8]:
for i in range ( 1001,19601,100 ) :
    url = base_url + str(i) + "-" + str(i + 99) + extension
    # print(url)
    
    ID = i
    hash = hashlib.md5(url.encode()).hexdigest()
    # print(hash)
    res = cur.execute("SELECT text FROM url_cache WHERE `hash` = '" + hash + "'" )
    text = res.fetchone()[0]
    
    # print(len(text))
    matches = re.findall(r"<SPAN .*\n.*\n.*\n.*\n.*\n.*\n.*<P>",text)
    
    for match in matches :
        # Found some matches
        # print( match )
        groups = re.findall(r"<SPAN CLASS='Number'>(.*)\.</SPAN><BR>\n.*\n(.*)<BR>\n(.*)<BR>\n(.*)<BR>\n.*\n.*>--(.*)</ADDRESS><P>", match)[0]
        
        if len( groups) == 5 :
            # https://stackoverflow.com/a/4869782

            poem_values = [ int(groups[0]), 
                           re.sub('<[^>]*>', '', groups[1]), 
                           re.sub('<[^>]*>', '', groups[2]),
                           re.sub('<[^>]*>', '', groups[3]),
                           re.sub('<[^>]*>', '', groups[4]),]
            # print(poem_values)

            insert_results = cur.execute("INSERT OR REPLACE INTO poems VALUES ( ?, ?, ?, ?, ? )", poem_values )
            # print(insert_results)
        else :
            pass
            # print(len(groups))
    # break

con.commit()

## Syllable Counter

In [10]:
dic = pyphen.Pyphen(lang='en')
# dic.inserted(text)

In [11]:
count = syllapy.count('additional')

In [140]:
syllable_counts = defaultdict(list)
invalid_words = list()
all_words = list()

In [141]:
for poem in cur.execute("SELECT * FROM poems;"):
    first_line = 0
    second_line = 0
    third_line = 0

    for word in re.sub(r'[^\w\s]', '', poem[1]).split() :
        all_words.append(word.lower())
        first_line += syllapy.count(word)
        # record bad words
        if syllapy.count(word) == 0 :
            invalid_words.append(word)
    for word in re.sub(r'[^\w\s]', '', poem[2]).split() :
        second_line += syllapy.count(word)
        all_words.append(word.lower())
    for word in re.sub(r'[^\w\s]', '', poem[3]).split() :
        third_line += syllapy.count(word)
        all_words.append(word.lower())
    
#     print(first_line)
#     print(second_line)
#     print(third_line)
    syllable_counts[poem[0]] = [first_line, second_line, third_line]
    
    

In [106]:
# syllable_counts.items()

# Analysis

In [99]:
first_lines = []
second_lines = []
third_lines = []

valid = 0

semi_valid = 0


for poem in syllable_counts.items():
    first_lines.append(poem[1][0])
    second_lines.append(poem[1][1])
    third_lines.append(poem[1][2])
    # break
    # print(type(poem[1][0]))

    if poem[1][0] == 5 and \
    poem[1][1] == 7 and \
    poem[1][2] == 5 :
        valid = valid + 1
        
    if poem[1][0] >= 4 and poem[1][0] <= 6 and \
    poem[1][1] >= 6 and poem[1][1] <= 8 and \
    poem[1][2] >= 4 and poem[1][2] <= 6 :
        semi_valid = semi_valid + 1
    
print("Sums of all syllables")
print(sum(first_lines))  
print(sum(second_lines))  
print(sum(third_lines))  

print("Mins of all syllables")
print(min(first_lines))  
print(min(second_lines))  
print(min(third_lines))  


print("Maxs of all syllables")
print(max(first_lines))  
print(max(second_lines))  
print(max(third_lines))  

print("Avg of all syllables")
print(round(mean(first_lines),4))  
print(round(mean(second_lines),4))  
print(round(mean(third_lines),4))  

print("Valid Poems")
print( str(valid)+" out of 19601" )
print( str(100*round(valid/19601,6))+"% are valid" )

print("Semi Valid Poems")
print( str(semi_valid)+" out of 19601" )
print( str(100*round(semi_valid/19601,6))+"% are semi valid" )



Sums of all syllables
93273
131057
93681
Mins of all syllables
0
0
0
Maxs of all syllables
8
77
22
Avg of all syllables
5.0147
7.0461
5.0366
Valid Poems
11299 out of 19601
57.645% are valid
Semi Valid Poems
17795 out of 19601
90.7862% are semi valid


## Naughty Words

In [114]:
print(invalid_words[:15])

['95', 'NC-17:', "'40s", '8th', "2's", '13:', '/', '13', '5', '1st', 'MST3K:', '1984:', 'B-52', '486:', '911!']


## Most Common Words

In [143]:
# Pass the split_it list to instance of Counter class.
AllWordsCounter = Counter(all_words)
  
# most_common() produces k frequently encountered
# input values and their respective counts.
print("Most Common All Words")
print(AllWordsCounter.most_common(20))

print()

filtered_words = [t for t in all_words if not t in stopwords.words("english")]

# Pass the split_it list to instance of Counter class.
FilteredWordsCounter = Counter(filtered_words)
  
# most_common() produces k frequently encountered
# input values and their respective counts.
print("Most Common Filtered Words")
print(FilteredWordsCounter.most_common(20))


Most Common All Words
[('spam', 15338), ('the', 8381), ('a', 4836), ('i', 4630), ('of', 4357), ('in', 3672), ('and', 3624), ('is', 3146), ('to', 3099), ('my', 3080), ('it', 2924), ('you', 2218), ('can', 2152), ('on', 1881), ('for', 1812), ('pink', 1654), ('with', 1579), ('me', 1365), ('eat', 1308), ('not', 1193)]

Most Common Filtered Words
[('spam', 15338), ('pink', 1654), ('eat', 1308), ('like', 1100), ('meat', 1070), ('one', 655), ('blue', 605), ('love', 586), ('pig', 578), ('spamku', 507), ('hormel', 503), ('good', 477), ('pork', 473), ('food', 473), ('dont', 463), ('oh', 463), ('new', 390), ('man', 387), ('life', 385), ('haiku', 377)]


# Sample of Poems

In [144]:
for poem in cur.execute("SELECT * FROM poems ORDER BY random() LIMIT 10;"):
    print(poem[1])
    print(poem[2])
    print(poem[3])
    print()

my mom brings home SPAM
and i wonder what it is
don't like the color

SPAM and gossiping
Are my twin pastimes of late
Eat and ruin lives

Turkey-shaped SPAM for
Thanksgiving dinner. None give
thanks but the turkey.

Helping out the poor
I offered the needy SPAM
Bruises are healed now

So many stories
Sadness and joy of mankind
Mingled in blue can

General Motors
Comes out with SPAM-powered car.
Total sales: Zero.

enigmatic brick
Shackelton wished he had some
expedition treat

Encased in pectin...
On that final sunny trot
cold SPAM dwells in bliss.

Sold Microsoft stock.
With money, bought SPAM futures.
The fools call me mad!

SPAM is great camping.
Fry it up in the pan with
Leftover fish guts.



In [157]:
con.close()